# Ollama Haystack integration demo
https://github.com/deepset-ai/haystack-integrations/blob/main/integrations/ollama.md

WORKING: ollama-haystack env

In [4]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [5]:
#!pip install ollama-haystack

In [6]:
from haystack_integrations.components.generators.ollama import OllamaGenerator

In [7]:
document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="""Haystack offers various components, each performing different kinds of tasks. You can see the whole variety in the PIPELINE COMPONENTS section in the left-side navigation. These are often powered by the latest Large Language Models (LLMs) and transformer models. Code-wise, they are Python classes with methods you can directly call. Most commonly, all you need to do is initialize the component with the required parameters and then run it with a run() method.
Working on this level with Haystack components is a hands-on approach. Components define the name and the type of all of their inputs and outputs. The Component API reduces complexity and makes it easier to create custom components, for example, for third-party APIs and databases. Haystack validates the connections between components before running the pipeline and, if needed, generates error messages with instructions on fixing the errors. """,
                meta={"heading":"Components"}
                ),
        Document(content="""Generators are responsible for generating text responses after you give them a prompt. They are specific for each LLM technology (OpenAI, Cohere, local models, and others). There are two types of Generators: chat and non-chat:
    The chat ones enable chat completion and are designed for conversational contexts. It expects a list of messages to interact with the user.
    The non-chat Generators use LLMs for simpler text generation (for example, translating or summarizing text).
Read more about various Generators in our guides. """,
                meta={"heading":"Generators"}
                ),
        Document(content="""Retrievers go through all the Documents in a Document Store, select the ones that match the user query, and pass it on to the next component. There are various Retrievers that are customized for specific Document Stores. This means that they can handle specific requirements for each database using customized parameters.
For example, for Elasticsearch Document Store, you will find both the Document Store and Retriever packages in its GitHub repo.  """,
                meta={"heading":"Retrievers"}),
        Document(content="""Document Store is an object that stores your Documents in Haystack, like an interface to a storage database. It uses specific functions like write_documents() or delete_documents() to work with data. Various components have access to the Document Store and can interact with it by, for example, reading or writing Documents.
If you are working with more complex Pipelines in Haystack, you can use a DocumentWriter component to write data into Document Stores for you.""",
                meta={"heading":"Document Stores"}),
        Document(content="""You can use different data classes in Haystack to carry the data through the system. The data classes are mostly likely to appear as inputs or outputs of your Pipelines.

Document class contains information to be carried through the Pipeline. It can be text, metadata, tables, or binary data. Documents can be written into Document Stores but also written and read by other components.

Answer class holds not only the answer generated in a Pipeline but also the originating query and metadata. """,
                meta={"heading":"Data Classes"}),
    ]
)

5

In [8]:
template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

In [9]:
pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="orca-mini", url="http://minti9:11434/api/generate"))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [10]:
query = "What are haystack retrievers?"

response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

print(response["llm"]["replies"])

Ranking by BM25...:   0%|          | 0/5 [00:00<?, ? docs/s]

[' Haystack Retrievers are a specific type of component in the Haystack framework that is used to retrieve documents from a specific Document Store based on a user query. They can be customized for different database systems and use customized parameters to handle specific requirements for each database.']
